In [1]:
import numpy as np
import pandas as pd
import math

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB2 #260x260
import tensorflow.keras.layers as layers
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.utils import Sequence
import imgaug.augmenters as iaa
import imgaug

from sklearn.model_selection import train_test_split

2023-01-19 14:53:05.205252: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data_dir = '/run/user/1000/gvfs/smb-share:server=titan.local,share=datasets/kitchenware-classification'
seed = 42

In [3]:
df_train_full = pd.read_csv(data_dir+'/train.csv', dtype={'Id': str})
df_extra = pd.read_csv(data_dir+'/data.csv', dtype={'Id': str})
df_train_full = pd.concat([df_train_full, df_extra], ignore_index = True)
df_train_full['filename'] = data_dir+'/images/' + df_train_full['Id'] + '.jpg'
df_train_full.head()

,Id,label,filename
0,0560,glass,/run/user/1000/gvfs/smb-share:server=titan.loc...
1,4675,cup,/run/user/1000/gvfs/smb-share:server=titan.loc...
2,0875,glass,/run/user/1000/gvfs/smb-share:server=titan.loc...
3,4436,spoon,/run/user/1000/gvfs/smb-share:server=titan.loc...
4,8265,plate,/run/user/1000/gvfs/smb-share:server=titan.loc...


In [4]:
image_width = 260
image_height = 260
NUM_CLASSES = 6
batch_size = 32
n_epochs = 12
learning_rate=1e-4

In [5]:
df_train, df_val, y_train, y_val = train_test_split(df_train_full, df_train_full['label'], test_size = 0.2, shuffle = True, stratify = df_train_full['label'], random_state = seed)
df_train

,Id,label,filename
7592,eu.95a129ef-d94a-4ab0-a4e4-e66aac61e55f,fork,/run/user/1000/gvfs/smb-share:server=titan.loc...
7704,eu.b90f170d-c2d2-47bf-bd61-375f9032e98f,cup,/run/user/1000/gvfs/smb-share:server=titan.loc...
7840,eu.84a95cb4-6026-40ed-bfb6-936d297a7d91,cup,/run/user/1000/gvfs/smb-share:server=titan.loc...
5030,5485,fork,/run/user/1000/gvfs/smb-share:server=titan.loc...
7114,eu.edfa5512-6f5d-4cf7-bbcc-a31eace9b484,spoon,/run/user/1000/gvfs/smb-share:server=titan.loc...
...,...,...,...
5033,5725,plate,/run/user/1000/gvfs/smb-share:server=titan.loc...
4172,5120,plate,/run/user/1000/gvfs/smb-share:server=titan.loc...
4507,9013,cup,/run/user/1000/gvfs/smb-share:server=titan.loc...
3625,2070,cup,/run/user/1000/gvfs/smb-share:server=titan.loc...


In [6]:
train_aug = iaa.Sequential(
    [
        #iaa.Resize({"height": image_height, "width": image_width}, interpolation=imgaug.ALL),
        iaa.Fliplr(0.3),
        iaa.Flipud(0.3), # vertically flip 20% of all images
        # Make some images brighter and some darker.
        # In 20% of all cases, we sample the multiplier once per channel,
        # which can end up changing the color of the images.
        #iaa.Sometimes(0.3,iaa.Multiply((0.8, 1.2), per_channel=0.2)),
        # `Sometimes()` applies a function randomly to the inputs with
        # a given probability (0.3, in this case).
        iaa.Sometimes(0.1, iaa.Cutout()),
        iaa.Sometimes(0.1, iaa.GaussianBlur(sigma=(0, 0.5))),
        #iaa.Sometimes(0.1, iaa.Cartoon(blur_ksize=3, segmentation_size=1.0, saturation=2.0, edge_prevalence=1.0))
    ],
    name = "train_aug"
)

In [7]:
def build_model(num_classes):
    
    inputs = layers.Input(shape = (image_width, image_height, 3))
    x=inputs
    model = EfficientNetB2(include_top=False, input_tensor=x, weights="imagenet")
    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(NUM_CLASSES, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    
    return model

In [8]:
train_datagen = ImageDataGenerator(preprocessing_function=train_aug.augment_image)

train_generator = train_datagen.flow_from_dataframe(
    df_train,
    x_col='filename',
    y_col='label',
    target_size=(image_width, image_height),
    batch_size=batch_size,
    rescale=1/255.
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_generator = val_datagen.flow_from_dataframe(
    df_val,
    x_col='filename',
    y_col='label',
    target_size=(image_width, image_height),
    batch_size=batch_size,
    rescale=1/255.,
)

Found 6390 validated image filenames belonging to 6 classes.
Found 1598 validated image filenames belonging to 6 classes.


In [9]:
model = build_model(num_classes=NUM_CLASSES)

history = model.fit(
    train_generator,
    epochs=n_epochs,
    validation_data=val_generator
)

2023-01-19 14:53:17.955171: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-19 14:53:18.002958: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-19 14:53:18.003174: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-19 14:53:18.003551: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

Epoch 1/12


2023-01-19 14:53:25.722106: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401


  1/200 [..............................] - ETA: 20:06 - loss: 2.3908 - accuracy: 0.1250

2023-01-19 14:53:26.591542: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-01-19 14:53:26.592060: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-01-19 14:53:26.592070: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-01-19 14:53:26.592823: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-01-19 14:53:26.592859: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2023-01-19 14:53:26.617363: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged onc

200/200 [==============================] - 138s 665ms/step - loss: 1.2338 - accuracy: 0.5723 - val_loss: 0.5728 - val_accuracy: 0.8730
Epoch 2/12
200/200 [==============================] - 135s 676ms/step - loss: 0.4809 - accuracy: 0.8416 - val_loss: 0.2768 - val_accuracy: 0.9180
Epoch 3/12
200/200 [==============================] - 133s 664ms/step - loss: 0.3463 - accuracy: 0.8905 - val_loss: 0.2208 - val_accuracy: 0.9312
Epoch 4/12
200/200 [==============================] - 133s 664ms/step - loss: 0.2906 - accuracy: 0.9023 - val_loss: 0.1971 - val_accuracy: 0.9355
Epoch 5/12
200/200 [==============================] - 128s 641ms/step - loss: 0.2560 - accuracy: 0.9192 - val_loss: 0.1849 - val_accuracy: 0.9393
Epoch 6/12
200/200 [==============================] - 137s 688ms/step - loss: 0.2451 - accuracy: 0.9211 - val_loss: 0.1750 - val_accuracy: 0.9424
Epoch 7/12
200/200 [==============================] - 130s 649ms/step - loss: 0.2186 - accuracy: 0.9285 - val_loss: 0.1689 - val_accura

In [11]:
df_test = pd.read_csv(data_dir+'/test.csv', dtype={'Id': str})
df_test['filename'] = data_dir+'/images/' + df_test['Id'] + '.jpg'
df_test.head()

,Id,filename
0,0678,/run/user/1000/gvfs/smb-share:server=titan.loc...
1,3962,/run/user/1000/gvfs/smb-share:server=titan.loc...
2,9271,/run/user/1000/gvfs/smb-share:server=titan.loc...
3,5133,/run/user/1000/gvfs/smb-share:server=titan.loc...
4,8842,/run/user/1000/gvfs/smb-share:server=titan.loc...


In [12]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_dataframe(
    df_test,
    x_col='filename',
    class_mode='input',
    target_size=(image_width, image_height),
    batch_size=batch_size,
    shuffle=False
)

Found 3808 validated image filenames.


In [13]:
y_pred = model.predict(test_generator)

119/119 [==============================] - 82s 680ms/step


In [14]:
classes = np.array(list(train_generator.class_indices.keys()))
classes

array(['cup', 'fork', 'glass', 'knife', 'plate', 'spoon'], dtype='<U5')

In [15]:
predictions = classes[y_pred.argmax(axis=1)]

In [16]:
df_submission = pd.DataFrame()
df_submission['filename'] = test_generator.filenames
df_submission['label'] = predictions

df_submission['Id'] = df_submission.filename.str[len(data_dir+'/images/'):-4]
del df_submission['filename']

In [17]:
df_submission[['Id', 'label']].to_csv('submissions/effb2_imnet_v4_aug.csv', index=False)

In [18]:
!kaggle competitions submit kitchenware-classification -f submissions/effb2_imnet_v4_aug.csv -m 'validation: 0.9481'

100%|██████████████████████████████████████| 38.8k/38.8k [00:02<00:00, 17.6kB/s]
Successfully submitted to Kitchenware Classification